<a href="https://colab.research.google.com/github/playeredlc/treinamento-h2ia/blob/master/Buscas/buscas_com_informa%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# O Problema
Sliding Puzzle - Bloco Deslizante

In [1]:
# !wget -qq https://miro.medium.com/max/700/1*W7jg4GmEjGBypd9WPktasQ.gif
from IPython.display import Image
Image(url='https://miro.medium.com/max/700/1*W7jg4GmEjGBypd9WPktasQ.gif',width=200)

# Resolver o quebra-cabeças usando Buscas

In [2]:
import numpy as np
import heapq

## Definição do nodo

Para a solução com busca informada, por meio do algoritmo A*, utiliza-se uma implementação do nodo semelhante a utilizada na [busca sem informação](https://github.com/playeredlc/treinamento-h2ia/blob/master/Buscas/buscas_sem_informa%C3%A7%C3%A3o.ipynb), porém, com algumas informações extras:

$Nodo.g =$ É o **custo** de chegar da raíz até o Nodo.

$Nodo.h =$ É o **custo estimado** de chegar do Nodo até a solução do problema.

$Nodo.f =$ É o **custo estimado** da solução de menor custo, passando por Nodo.

O cálculo da estimativa do custo para ir de um determinado nodo até a solução  (Nodo.h) é dado pela soma de duas heurísticas:
* O número de peças fora da posição correta;
* O número de movimentos necessários para reposionar cada peça no lugar correto, sendo que outras peças não restrigem o seu movimento.

*heurísticas inspiradas na fonte: https://www.youtube.com/watch?v=AmDFYN45a3I&t=1068s*

In [3]:
class Node:
  def __init__(self, state, parent=None):
    self.state = state
    self.parent = parent
    self.path_to_root = []

    if(self.parent == None):
      self.g = 0
    else:
      self.g = self.parent.g + 1
    
    self.h = self.heuristic_dist()

    self.f = self.g + self.h
  
  def heuristic_dist(self):    
    final_st = np.array([[1, 2, 3],
                         [4, 5, 6],
                         [7, 8, 0]])    
    r, c = self.state.shape
    
    h1=0
    for r in range(r):
      for c in range(c):
        r2, c2 = self.get_coord(final_st[r, c])
        # number of moves to place in the right position
        dist = abs(r - r2) + abs(c - c2)
        if(dist != 0):
          # wrong position adds 1
           h1 += 1
        h1 += dist
    
    return h1

  def get_coord(self, value):
    coord = np.where(self.state == value)
    r, c = coord[0][0], coord[1][0]
    
    return r, c

  def get_zero(self):
    coord = np.where(self.state == 0)
    r, c = coord[0][0], coord[1][0]

    return r, c

  def move_left(self):
    r, c = self.get_zero()
    if(c != 0):
      new_st = self.state.copy()
      # swap
      new_st[r, c], new_st[r, c-1] = new_st[r, c-1], new_st[r, c]
      return new_st    
    else:
      return np.empty(0)
  
  def move_up(self):
    r, c = self.get_zero()
    if(r != 0):
      new_st = self.state.copy()
      # swap
      new_st[r, c], new_st[r-1, c] = new_st[r-1, c], new_st[r, c]
      return new_st    
    else:
      return np.empty(0)
  
  def move_right(self):
    r, c = self.get_zero()
    if(c < self.state.shape[1]-1):
      new_st = self.state.copy()
      # swap
      new_st[r, c], new_st[r, c+1] = new_st[r, c+1], new_st[r, c]
      return new_st    
    else:
      return np.empty(0)

  def move_down(self):
    r, c = self.get_zero()
    if(r < self.state.shape[0]-1):
      new_st = self.state.copy()
      # swap
      new_st[r, c], new_st[r+1, c] = new_st[r+1, c], new_st[r, c]
      return new_st
    else:
      return np.empty(0)

  def generate_children(self):
    children_list = []

    child_state = self.move_left()
    if(child_state.any()):
      children_list.append(Node(child_state, self))
    
    child_state = self.move_up()
    if(child_state.any()):
      children_list.append(Node(child_state, self))
    
    child_state = self.move_right()
    if(child_state.any()):
      children_list.append(Node(child_state, self))
    
    child_state = self.move_down()
    if(child_state.any()):
      children_list.append(Node(child_state, self))

    return children_list

  def generate_path(self):
    self.path_to_root.clear()
    previous = self.parent
    while(previous.parent != None):
      self.path_to_root.append(previous.state.copy())
      previous = previous.parent
    
    return self.path_to_root

  def print_solution(self):
    move = 1
    for state in reversed(self.path_to_root):
      print(f'\nMove {move}:\n{state}')
      move += 1
    print(f'\nFinal move:\n{self.state}')

## Busca A*

Na busca A*, utiliza-se o valor de `Nodo.f` (custo estimado da solução de menor custo, passando por Nodo.) para priorizar os nodos durante a busca. A estrutura utilizada para armazenar os nodos a serem explorados é uma heap (minheap).

Caso a heuristica `Nodo.h` satisfaça algumas condições (admissibilidade/consistência) a busca é completa e ótima.

*Admissibilidade:* Uma heurística admissível é aquela que nunca superestima o custo de atingir o objetivo. É uma estimativa otimista por natureza.

*Consistência:* A consistência é necessária para busca em grafos. Uma heurística consistente é aquela onde, para cada **nodo n** e **nodo n'** (filho de n), o custo de chegar a estimativa de *n* $(n.h)$ não pode ser maior do que o custo de chegar a *n'* mais o custo de chegar a estimativa de *n'* $(n'.f)$.

In [4]:
# https://docs.python.org/3/library/heapq.html#

final_state = np.array([[1, 2, 3],
                        [4, 5, 6],
                        [7, 8, 0]])

def a_star_search(initial_state, final_state, verbose=False):
  it = 0
  entry_count = 1
  hq = []
  visited_states = set()  

  root = Node(initial_state)
  heapq.heappush(hq, (root.f, entry_count, root))

  while(len(hq) != 0):
    
    if(verbose):        
      print(f'\nIteration {it} ...')
      print(f'Visited states amount: {len(visited_states)}')
    
    # pop the node with lowest 'f' from heap
    node = heapq.heappop(hq)[2]

    if(np.array_equal(node.state, final_state)):      
      print(f'\nFinal iteration {it}.')
      print(f'Visited states amount: {len(visited_states)}')
      print(f'\nRESULT FOUND!\n{node.state}\n')      
      
      node.generate_path()
      return node
    
    children_list = node.generate_children()

    for child in children_list:
      if((not (np.array2string(child.state)) in visited_states)):
        heapq.heappush(hq, (child.f, entry_count, child))
        entry_count += 1
    
    visited_states.add(np.array2string(node.state))
    it += 1

  return False


### Teste com a instância mais difícil do problema

Quando foram desenvolvidos as [buscas sem informação](https://github.com/playeredlc/treinamento-h2ia/blob/master/Buscas/buscas_sem_informa%C3%A7%C3%A3o.ipynb) (dfs e bfs), realizou-se a busca da solução para a [instância mais difícil do problema](http://w01fe.com/blog/2009/01/the-hardest-eight-puzzle-instances-take-31-moves-to-solve/). Nenhum dos algoritmos obteve uma boa solução para o problema.

A busca em largura encontrou a solução ótima, porém, levou aproximadamente 15 minutos para encontrá-la. Já a busca em profundidade levou menos de um minuto, porém, a solução encontrada possui um custo de mais de 58 mil movimentos, enquanto a solução ótima possui 31.

Como pode ser observado abaixo, o algoritmo A* obteve uma performance muito superior para o problema, encontrando a solução ótima para essa instância do problema em aproximadamente 5 segundos.

`CPU times: user 5.16 s, sys: 264 ms, total: 5.42 s
Wall time: 4.9 s`

In [7]:
%%time

initial_state = np.array([[8, 6, 7],
                          [2, 5, 4],
                          [3, 0, 1]])

res = a_star_search(initial_state, final_state, verbose=False)

cost = len(res.path_to_root)+1
print(f'Solution cost: {cost}\n')


Final iteration 10343.
Visited states amount: 9688

RESULT FOUND!
[[1 2 3]
 [4 5 6]
 [7 8 0]]

Solution cost: 31

CPU times: user 5.16 s, sys: 264 ms, total: 5.42 s
Wall time: 4.9 s


#### Passos da solução

In [8]:
print(f'Initial state:\n{initial_state}')
res.print_solution()

Initial state:
[[8 6 7]
 [2 5 4]
 [3 0 1]]

Move 1:
[[8 6 7]
 [2 5 4]
 [3 1 0]]

Move 2:
[[8 6 7]
 [2 5 0]
 [3 1 4]]

Move 3:
[[8 6 0]
 [2 5 7]
 [3 1 4]]

Move 4:
[[8 0 6]
 [2 5 7]
 [3 1 4]]

Move 5:
[[0 8 6]
 [2 5 7]
 [3 1 4]]

Move 6:
[[2 8 6]
 [0 5 7]
 [3 1 4]]

Move 7:
[[2 8 6]
 [3 5 7]
 [0 1 4]]

Move 8:
[[2 8 6]
 [3 5 7]
 [1 0 4]]

Move 9:
[[2 8 6]
 [3 5 7]
 [1 4 0]]

Move 10:
[[2 8 6]
 [3 5 0]
 [1 4 7]]

Move 11:
[[2 8 6]
 [3 0 5]
 [1 4 7]]

Move 12:
[[2 8 6]
 [0 3 5]
 [1 4 7]]

Move 13:
[[2 8 6]
 [1 3 5]
 [0 4 7]]

Move 14:
[[2 8 6]
 [1 3 5]
 [4 0 7]]

Move 15:
[[2 8 6]
 [1 3 5]
 [4 7 0]]

Move 16:
[[2 8 6]
 [1 3 0]
 [4 7 5]]

Move 17:
[[2 8 6]
 [1 0 3]
 [4 7 5]]

Move 18:
[[2 0 6]
 [1 8 3]
 [4 7 5]]

Move 19:
[[2 6 0]
 [1 8 3]
 [4 7 5]]

Move 20:
[[2 6 3]
 [1 8 0]
 [4 7 5]]

Move 21:
[[2 6 3]
 [1 0 8]
 [4 7 5]]

Move 22:
[[2 0 3]
 [1 6 8]
 [4 7 5]]

Move 23:
[[0 2 3]
 [1 6 8]
 [4 7 5]]

Move 24:
[[1 2 3]
 [0 6 8]
 [4 7 5]]

Move 25:
[[1 2 3]
 [4 6 8]
 [0 7 5]]

Move 26:
[[1 2 

## Discorra sobre o desempenho dos métodos em questões de:
1.   Consumo de memória
2.   Processamento

Como foi discutido na seção de teste, o algoritmo A* obteve uma peformance muito superior aos dois algoritmos de busca cega para uma instância muito difícil do problema, isso se deve ao uso de uma boa heurística.

Ainda assim, é necessário armazenar todos os nós gerados em memória e dependendo da escala do problema, pode haver um estouro de memória. Comparando com a busca em largura, a profundidade da busca é a mesma, ou seja, aquele nível em que a solução ótima se encontra, porém, muitos nodos não são gerados diminuindo a complexidade espacial.

A utilização da heurística também tem um impacto positivo no tempo de processamento, isso porque a informação extra evita que nodos que se distanciam da solução sejam explorados. Na busca em profundidade, por exemplo, isso não acontece e muitas vezes ramos muito profundos que não levam a solução devem ser processados.